## STAYSL Post-Processing Tools

Tools to rip and plot modeled and unfolded neutron spectra.

First, import required modules and functions:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#-------------------------------------------------------------------------------------------------------------#
def difFluxToFlux(binE,dF):
    """
    Converts from differential flux to flux. Valid only for binned data with mid-point values (the 
    integration method uses the mid-point rule).
    
    Parameters
    ==========
    binE : list of floats
        The midpoint, lower, or upper bin energies
    dF: list of floats
        The differential flux in each bin

    Optional
    ========
        
    Returns
    =======
    flux : list of floats
        The flux in each bin
    """ 
    
    flux=[]
    
    flux.append(dF[0]*binE[0])
    for i in range(1,len(dF)):
        flux.append(dF[i]*(binE[i]-binE[i-1]))
        
    return flux

#-------------------------------------------------------------------------------------------------------------#
def normFlux(binE,f):
    """
    Normalized a spectrum so that it produces a PDF. Valid only for histogram binned data.
   
    Parameters
    ==========
    binE : list of floats
        The bin energies edges
    f: list of floats
        The flux in each bin

    Optional
    ========
        
    Returns
    =======
    nF : list of floats
        The normalized flux or differential flux in each bin
    """ 
    
    nF=[]
    
    nF.append(f[0]*binE[0])
    for i in range(1,len(f)):
        nF.append(f[i])
    
    nF=np.asarray(nF)
    
    return nF/sum(nF)

#-------------------------------------------------------------------------------------------------------------#
def buildHisto(midE,f):
    """
    Builds up the edges in energy and flux for plotting a histogram from a list of mid bin energies and flux values. 
   
    Parameters
    ==========
    midE : list of floats
        The midpoint bin energies
    f: list of floats
        The flux or differential flux in each bin

    Optional
    ========
        
    Returns
    =======
    binEdges : list of floats
        The upper and lower edges for each bin
    edgeFlux : list of floats
        The flux at each bin edge
    """ 
    
    binEdges=[]
    edgeFlux=[]
    binEdges.append(-1.0)
    edgeFlux.append(f[0])
    for i in range(1,len(f)):
        binEdges.append((midE[i-1]+midE[i])/2)
        edgeFlux.append(f[i-1])
        binEdges.append((midE[i-1]+midE[i])/2)
        edgeFlux.append(f[i])

    binEdges.append(midE[-1]+(midE[-1]+midE[-2])/2)
    edgeFlux.append(f[-1])
    binEdges.append(midE[-1]+(midE[-1]+midE[-2])/2)
    edgeFlux.append(0.0)

    return binEdges, edgeFlux

### User Inputs
Specify the file locations:

In [2]:
#outPath="C:/Users/James/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Simulated/88Inch_Activation/88_Vault/88_Run/STAYSL_PNNL/88_Run/"
outPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Data/Simulated/88Inch_Activation/88_Vault_ETA_IRDFF/STAYSL_PNNL/88_Run/"
outName="sta_xfr.dat"

### Read in the unfolded output
Not, the same output file has the "unadjusted" flux (i.e. the input spectrum)

In [3]:
eBins=[]
unadjFlux=[]
adjFlux=[]
# Open file
try: 
    f = open(outPath+outName, 'r') 
    # Skip header lines
    line=f.next()

    # Read the file line by line and store the values 
    for line in f:
        split_list=line.strip().split('\t')
        eBins.append(float(split_list[0]))
        unadjFlux.append(float(split_list[1]))
        adjFlux.append(float(split_list[2]))

    # Close the file
    f.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror) 

### Calculate the relative difference between original and unfolded spectrum

In [4]:
# Convert to numpy arrays
unadjFlux=np.asarray(unadjFlux)
adjFlux=np.asarray(adjFlux)

# Normalize such that the area under the curve=1 (generate PDF)
unadjFlux=normFlux(eBins,unadjFlux)
adjFlux=normFlux(eBins,adjFlux)

print "Normalization check (should both equal one:", sum(adjFlux), sum(adjFlux)

relDiff=abs(unadjFlux-adjFlux)/adjFlux * 100
relDiff=np.nan_to_num(relDiff)

Normalization check (should both equal one: 1.0 1.0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in divide


### Plot

In [8]:
# Allow use of Tex sybols
plt.rc('text', usetex=True)

# Set up figure
fig = plt.figure()
ax1 = fig.add_axes([0.15, 0.1, 0.7, 0.85])

# Preset data set format scheme
s=10
linewidth=['2.5']
marker=['.','o','v','^','<','>','1','2','3','4','8','s','p','*','h','H','+','x','d','D']
linestyle=['-','--','-']
dashes=[[10, 0.1],[10, 5, 10, 5],[10,3,1,3]]

# Set Line color cycle
ax1.set_color_cycle(['k', 'k', 'k'])

# Set axes
ax1.axis([1E-3, 20, np.min(adjFlux), np.max(adjFlux)])
ax1.axis([1E-3, 20, 1E-5, 1])
ax1.set_xscale('log')
ax1.set_yscale('log')

# Set axes labels and plot title.
#ax1.set_title('\\textbf{NIF Simulated Experimental Results}', fontsize=18, weight="bold")    
ax1.set_xlabel('\\textbf{Energy [MeV]}', fontsize=18, weight="bold")
ax1.set_ylabel('\\textbf{Normalized Flux [n cm$^{-2}$ s$^{-1}$]}', fontsize=18, weight="bold")
ax1.tick_params(axis='both', which='major', labelsize=18, width=2)
ax1.tick_params(axis='both', which='minor', width=2)

# Add data set to plot
ax1.plot(eBins, adjFlux, linewidth=linewidth[0], linestyle=linestyle[0], 
         marker=None,label="STAYSL Unfold", dashes=dashes[0]) 
ax1.plot(eBins, unadjFlux, linewidth=linewidth[0], linestyle=linestyle[1], 
         label="ETA Modeled", dashes=dashes[1]) 

# Add arrows to show which axis data corresponds to
#w/o ETA
#ax1.annotate("",
#            xy=(4.5,0.025), xycoords='data',
#            xytext=(6.5, 0.025), textcoords='data',
#            arrowprops=dict(arrowstyle="simple,head_length=0.5,head_width=0.5,tail_width=0.2",
#                connectionstyle="angle3,angleA=90,angleB=0",color="k"))
#ax1.annotate("",
#            xy=(12, 0.002), xycoords='data',
#            xytext=(10, 0.003), textcoords='data',
#            arrowprops=dict(arrowstyle="simple,head_length=0.5,head_width=0.5,tail_width=0.2",
#                connectionstyle="angle3,angleA=90,angleB=0",color="k"))
#w/ ETA
ax1.annotate("",
            xy=(0.012,0.0015), xycoords='data',
            xytext=(0.04, 0.0015), textcoords='data',
            arrowprops=dict(arrowstyle="simple,head_length=0.5,head_width=0.5,tail_width=0.2",
                connectionstyle="angle3,angleA=90,angleB=0",color="k"))
ax1.annotate("",
            xy=(0.9, 0.0015), xycoords='data',
            xytext=(0.4, 0.003), textcoords='data',
            arrowprops=dict(arrowstyle="simple,head_length=0.5,head_width=0.5,tail_width=0.2",
                connectionstyle="angle3,angleA=90,angleB=0",color="k"))

# Add and locate legend
line=ax1.plot(0, 0, linewidth=linewidth[0], linestyle=linestyle[2], 
         marker=None,label="Difference", dashes=dashes[2])[0] #Kludge

leg = ax1.legend()
plt.legend(borderaxespad=1.00, loc=1, fontsize=16, handlelength=5, borderpad=0.5,\
            labelspacing=0.75, fancybox=True, framealpha=0.5, numpoints=1);

# Add secondary axis
ax2 = ax1.twinx()
ax2.axis([1E-3, 20, -10, 40])
ax2.axis([1E-3, 20, -np.max(relDiff)*2, np.max(relDiff)*2])
ax2.set_xscale('log')
#ax2.set_yscale('log')

# Set axes labels and plot title.
#ax2.set_xlabel('\\textbf{Energy [MeV]}', fontsize=14)
ax2.set_ylabel('\\textbf{Percent [$\%$]}', fontsize=18, weight="bold")
ax2.tick_params(axis='both', which='major', labelsize=18, width=2)
ax2.tick_params(axis='both', which='minor', width=2)
ax2.set_color_cycle(['k', 'k', 'k'])
 
# Plot Data
ax2.plot(eBins, relDiff, linewidth=linewidth[0], linestyle=linestyle[2], 
         marker=None,label="Difference", dashes=dashes[2]) 

plt.show()
fig.savefig(outPath+'results_comp_NIFfoils', bbox_inches='tight')